In [5]:
import os
import time
import queue
import datetime
import requests
import threading
import numpy as np
import pandas as pd
import tkinter as tk
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from keras.models import Sequential
from keras.layers.core import Dense
from keras.models import load_model
now = datetime.datetime.now()
if (now.day - 1) < 10:
    day = '0' + str(now.day - 1)
else:
    day = str(now.day - 1)
today = str(now.year) + '-' + str(now.month) + '-' + day
def predict(home, away):
    df = pd.read_excel('C:\\Users\\ecom\\Desktop\\NBAData\\trainingDataSet.xlsx')
    home_data = df.loc[(df.Tm == home) & (df.place == 'home'),].tail(5).mean()
    away_data = df.loc[(df.Tm == away) & (df.place == 'away'),].tail(5).mean()
    home_data = home_data.drop(['result', '+/-', 'GmSc'], axis=0)
    away_data = away_data.drop(['result', '+/-', 'GmSc'], axis=0)
    new_x = np.array(home_data - away_data)
    model1 = load_model('C:\\Users\\ecom\\Desktop\\nba-model.hdf5')
    return model1.predict_classes(new_x[np.newaxis,:])[0][0]

def predict_NBA():
    now = datetime.datetime.now()
    if now.day < 10:
        day = '0' + str(now.day - 1)
    else:
        day = str(now.day - 1)
    today = str(now.year) + '-' + str(now.month) + '-' + day
    path = 'C:\\Users\\ecom\\Desktop\\NBAData\\schedual\\' + str(now.year + 1) + '_season_schedual.xlsx'
    df = pd.read_excel(path)
    dftoday = df.loc[df.Date == today, ]
    com = []
    for i in range(len(dftoday)):
        com1 = [dftoday.iloc[i,3], dftoday.iloc[i,1]]
        com.append(com1)
    f = open('C:\\Users\\ecom\\Desktop\\NBAData\\Predict_Log.txt','a')
    text1.delete(1.0, tk.END)
    text1.insert(tk.END, today + '\n')
    text1.insert(tk.END, '--------------------------------------------\n')
    f.write('--------------------------------------------\n')
    for i in range(len(com)):
        p = predict(com[i][0], com[i][1])
        if p == 1:
            text1.insert(tk.END, '%s( win) vs %s(loss)\n' % (com[i][0], com[i][1]))
            f.write('%s( win) vs %s(loss)\n' % (com[i][0], com[i][1]))
        else:
            text1.insert(tk.END, '%s(loss) vs %s( win)\n' % (com[i][0], com[i][1]))
            f.write('%s( win) vs %s(loss)\n' % (com[i][0], com[i][1]))
    text1.insert(tk.END, '--------------------------------------------\n')
    text1.insert(tk.END, '輸贏自己負責\n')
    f.write('--------------------------------------------\n')
    f.close()

def datatraining():
    dfroot = pd.read_excel('C:\\Users\\ecom\\Desktop\\NBAData\\trainingDataSet.xlsx')
    dfroot = dfroot.reset_index(drop=True)
    dataX = dfroot.drop(['Date', 'Tm', 'place', 'Opp', 'result', '+/-', 'GmSc'], axis=1)
    dataY = dfroot['result']
    train_x = np.array(dataX)[::2]
    train_y = np.array(dataY)[::2] 
    test_x = np.array(dataX)[1::2]
    test_y = np.array(dataY)[1::2]
    #build model
    model = load_model('nba-model.hdf5')
    model.fit(train_x, train_y, batch_size=16, epochs=10)
    model.evaluate(test_x, test_y)
    model.save("C:\\Users\\ecom\\Desktop\\nba-model.hdf5")
    text1.delete(1.0, tk.END)
    text1.insert(tk.END, 'datatraining completed\n')
    
now = datetime.datetime.now()
#################change day
tday = now.day - 2
if tday < 10:
    day = '0' + str(tday)
else:
    day = str(tday)
month = now.month
if month < 8:
    year = str(now.year + 1)
    seasonyear = str(now.year)
else:
    year = str(now.year)
    seasonyear = str(now.year + 1)


def updateschedual():
    #讀取交易名單
    dfch = pd.read_html('https://www.cbssports.com/nba/transactions/')
    for ro in range(1,len(dfch)):
        dfch[0] = dfch[0].append(dfch[ro], ignore_index = True)
    dfch1 = dfch[0]
    newcol = ['Team', 'Player', 'Transaction']
    dfch1.columns = (newcol)
    for ro in range(len(dfch1)):
        n = ''
        wordcount = 0
        for ro1 in dfch1.iloc[ro,1]:
            if ro1 == ' ':
                wordcount = wordcount + 1
                if wordcount == 3:
                    n = ''
                else:
                    n = n + ro1
            else:
                n = n + ro1
        dfch1.iloc[ro,1] = n
    dfch1 = dfch1.drop(dfch1.loc[dfch1.Transaction == 'Waived'].index)
    dfch1 = dfch1.reset_index(drop=True)
    chdic = {}
    for re in range(len(dfch1)):
        chdic[dfch1.iloc[re,1]] = dfch1.iloc[re,0]
    East = ['MIA', 'DET', 'IND', 'ATL', 'WAS', 'NYK', 'CHI', 'CLE']
    West = ['DEN', 'GSW', 'MEM', 'HOU', 'MIN', 'DAL', 'UTA', 'SAC']
    side = [East, West]
    path = 'C:\\Users\\ecom\\Desktop\\NBAData\\2019\\'
    for i in range(len(side)):
        if i == 0:
            aside = 'East'
        else:
            aside = 'West'
        for j in side[i]:
            teampath = path + '\\' + aside + '\\' + j + '\\'
            listdata = os.listdir(teampath)
            for k in listdata:
                if k == 'Derrick Jones Jr..xlsx':
                    os.rename(teampath + k, teampath + 'Derrick Jones.xlsx')
                elif k == 'Glenn Robinson III.xlsx':
                    os.rename(teampath + k, teampath + 'Glenn Robinson.xlsx')
                elif k == 'DeVaughn Akoon-Purcell .xlsx':
                    os.rename(teampath + k, teampath + 'DeVaughn Akoon-Purcell.xlsx')
                elif k == 'Damion Lee .xlsx':
                    os.rename(teampath + k, teampath + 'Damion Lee.xlsx')
                elif k == 'Jaren Jackson Jr..xlsx':
                    os.rename(teampath + k, teampath + 'Jaren Jackson.xlsx')
                elif k == 'Edmond Sumner .xlsx':
                    os.rename(teampath + k, teampath + 'Edmond Sumner.xlsx')
                elif k == 'Gary Clark .xlsx':
                    os.rename(teampath + k, teampath + 'Gary Clark.xlsx')
                elif k == 'C.J. Williams .xlsx':
                    os.rename(teampath + k, teampath + 'C.J. Williams.xlsx')
                elif k == 'Alex Poythress .xlsx':
                    os.rename(teampath + k, teampath + 'Alex Poythress.xlsx')
                elif k == 'Dennis Smith Jr..xlsx':
                    os.rename(teampath + k, teampath + 'Dennis Smith.xlsx')
                elif k == 'Kelly Oubre Jr..xlsx':
                    os.rename(teampath + k, teampath + 'Kelly Oubre.xlsx')
                elif k == 'Troy Brown Jr..xlsx':
                    os.rename(teampath + k, teampath + 'Troy Brown.xlsx')
                elif k == 'Allonzo Trier .xlsx':
                    os.rename(teampath + k, teampath + 'Allonzo Trier.xlsx')
                elif k == 'Tim Hardaway Jr..xlsx':
                    os.rename(teampath + k, teampath + 'Tim Hardaway.xlsx')
                elif k == 'Wendell Carter Jr..xlsx':
                    os.rename(teampath + k, teampath + 'Wendell Carter.xlsx')
                elif k == 'Larry Nance Jr..xlsx':
                    os.rename(teampath + k, teampath + 'Larry Nance.xlsx')
                elif k == 'Naz Mitrou-Long .xlsx':
                    os.rename(teampath + k, teampath + 'Naz Mitrou-Long.xlsx')
                elif k == 'Taurean Prince.xlsx':
                    os.rename(teampath + k, teampath + 'Taurean Waller-Prince.xlsx')
                elif k == 'D.J. Stephens .xlsx':
                    os.rename(teampath + k, teampath + 'D.J. Stephens.xlsx')
                elif k == 'Duncan Robinson .xlsx':
                    os.rename(teampath + k, teampath + 'Duncan Robinson.xlsx')
                elif k == 'Marvin Bagley III.xlsx':
                    os.rename(teampath + k, teampath + 'Marvin Bagley.xlsx')
                elif k == 'Troy Williams .xlsx':
                    os.rename(teampath + k, teampath + 'Troy Williams.xlsx')
                elif k == 'Frank Mason III.xlsx':
                    os.rename(teampath + k, teampath + 'Frank Mason.xlsx')
    
    path = 'C:\\Users\\ecom\\Desktop\\NBAData\\schedual\\' + str(now.year + 1) + '_season_schedual.xlsx'
    com2 = pd.read_excel(path)
    dayformat = year + '-' + str(month) + '-' + day
    comday = com2.loc[com2.Date == dayformat, ]
    com = []
    for i in range(len(comday)):
        eachcom = [comday.iloc[i, 3], comday.iloc[i, 1]]
        com.append(eachcom)

    url = 'https://www.basketball-reference.com/boxscores/?month=' + str(month) + '&day=' + day + '&year=' + year
    text1.delete(1.0, tk.END)
    text1.insert(tk.END, '%s is updating\n' % path)
    try:
        table = pd.read_html(url)
        #get result
        todayres = []
        for i in range(0, len(com) * 3, 3):
            eachres = [table[i].iloc[1, 1], table[i].iloc[0, 1]]
            todayres.append(eachres)
        index = com2.loc[com2.Date == dayformat, ].index
        for i in range(len(index)):
            com2.iloc[index[i], 4] = todayres[i][0]
            com2.iloc[index[i], 2] = todayres[i][1]
        com2.to_excel('C:\\Users\\ecom\\Desktop\\NBAData\\schedual\\' + str(now.year + 1) + '_season_schedual.xlsx')
        text1.insert(tk.END, 'schedual has already update\n')
    except:
        text1.insert(tk.END, '%s not found!! Try later\n' % url)
    #updatedata    
    home = []
    for i in com:
        home.append(i[0])
    daystring = str(now.year) + str(month) + day + '0'
    demodf = pd.read_excel('C:\\Users\\ecom\\Desktop\\NBAData\\2019\\West\\DEN\\Gary Harris.xlsx')
    democol = demodf.columns
    logdf = pd.read_excel('C:\\Users\\ecom\\Desktop\\NBAData\\gamelog\\2019\\West\\DEN\\gamelog.xlsx')
    todaylog = pd.DataFrame(columns=logdf.columns)
    for i in home:
        url1 = 'https://www.basketball-reference.com/boxscores/' + daystring + i + '.html'
        dfgl = pd.read_html(url1)
        homelog = dfgl[2]
        awaylog = dfgl[0]
        alllog = [homelog, awaylog]
        for j in range(len(alllog)):
            #get team name in every game
            if j == 0:
                team = i
            else:
                for b in com:
                    if b[0] == i:
                        team = b[1]

            realcol = alllog[j].loc[alllog[j].Starters == 'FG',].values[0]
            realcol = realcol[0:21]
            realcol[0] = 'Starters'
            alllog[j] = alllog[j].drop(['PTS', '+/-'], axis=1)
            alllog[j].columns = (realcol)
            alllog[j] = alllog[j][alllog[j].Starters != 'Reserves']
            alllog[j] = alllog[j][alllog[j].Starters != 'Team Totals']
            alllog[j] = alllog[j].reset_index(drop=True)

            testdf = pd.DataFrame(alllog[j], columns=democol)
            testdf = testdf.drop('index', axis=1)
            testdf['Date'] = dayformat
            testdf['Unnamed: 5'] = '@'
            for l in range(len(com)):
                if com[l][0] == team:
                    gameres = todayres[l][0] - todayres[l][1]
                    if gameres > 0:
                        testdf['Unnamed: 7'] = 'W (+' + str(gameres) + ')'
                    else:
                        testdf['Unnamed: 7'] = 'L (' + str(gameres) + ')'
                    testdf['Tm'] = com[l][0]
                    testdf['Opp'] = com[l][1]
                elif com[l][1] == team:
                    gameres = todayres[l][0] - todayres[l][1]
                    if gameres > 0:
                        testdf['Unnamed: 7'] = 'W (+' + str(gameres) + ')'
                    else:
                        testdf['Unnamed: 7'] = 'L (' + str(gameres) + ')'
                    testdf['Tm'] = com[l][1]
                    testdf['Opp'] = com[l][0]
            teamside = os.listdir('C:\\Users\\ecom\\Desktop\\NBAData\\2019\\West')
            count = 0
            for z in teamside:
                if z == team:
                    count = count + 1
            if count > 0:
                side = 'West'
            else:
                side = 'East'
            totalpath = 'C:\\Users\\ecom\\Desktop\\NBAData\\' + seasonyear + '\\' + side + '\\' + team + '\\'
            dft = pd.read_excel(totalpath + 'total.xlsx')
            dftdate = dft.Date.max()
            for ind in dft.columns:
                if ind == 'index':
                    dft = dft.drop('index', axis=1)
            dft = dft.append(testdf, ignore_index=True)

            dfcor = pd.read_excel('C:\\Users\\ecom\\Desktop\\NBAData\\2013\\East\\ATL\\total.xlsx')
            colcor = dfcor.columns
            for chcol in colcor:
                if chcol == 'index':
                    colcor = colcor.drop('index')
            dft.columns = (colcor)
            dft.to_excel(totalpath + 'total.xlsx')
            text1.insert(tk.END, 'total has already update\n')
            
            #update personal data
            dropcol = ['Rk', 'G', 'Age', 'GS', 'MP']
            testdf = testdf.drop(dropcol, axis=1)
            newdf = pd.read_excel('C:\\Users\\ecom\\Desktop\\NBAData\\2019\\West\\LAL\\JaVale McGee.xlsx')
            newcol = newdf.columns
            
            for m in range(len(testdf)):
                playername = alllog[j].Starters[m]
                namecount = 0
                for chn in chdic.keys():
                    if playername == chn:
                        namecount = namecount + 1
                if namecount > 0:
                    dfr = pd.DataFrame(columns=newcol)
                else:
                    updatepath = totalpath + playername + '.xlsx'
                    dfr = pd.read_excel(updatepath)
                    dfrdate = dfr.Date.max()
                dfr = dfr.append(testdf.loc[m, ])
                dfr.to_excel(updatepath)
                text1.insert(tk.END, '%s has already update\n' % playername)

                cleanpath = 'C:\\Users\\ecom\\Desktop\\NBAData\\gamelog\\' + seasonyear + '\\' + side + '\\' + team + '\\gamelog.xlsx'
                dfcl = pd.read_excel(cleanpath)
                dfcldate = dfcl.Date.max()
                dfsum = testdf.drop(['Date', 'Tm', 'Unnamed: 5', 'Opp', 'Unnamed: 7'],axis=1)
                dfsum = dfsum.fillna(0)
                dfasum = pd.DataFrame(columns=dfsum.columns)
                dflist = []
                for sumcol in dfsum.columns:
                    dfsum[sumcol] = dfsum[sumcol].astype('float', copy=False)
                    dflist.append(dfsum[sumcol].sum())
                dfasum.loc[0, ] = dflist


                dflog = pd.DataFrame(dfasum, columns=['Date', 'Tm', 'Unnamed: 5', 'Opp', 'FG', 'FGA', 'FG%', '3P', '3PA',
                           '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK',
                           'TOV', 'PF', 'PTS', 'GmSc', '+/-', 'Unnamed: 7'])
                dflog.iloc[0, 0] = testdf.iloc[0, 0]
                dflog.iloc[0, 1] = testdf.iloc[0, 1]
                dflog.iloc[0, 2] = testdf.iloc[0, 2]
                dflog.iloc[0, 3] = testdf.iloc[0, 3]
                dflog.iloc[0, 24] = testdf.iloc[0, 4]

                a = ''
                dflog['Unnamed: 7'] = dflog['Unnamed: 7'].astype('str', copy=False)
                for n in dflog['Unnamed: 7'][0]:
                    if n == ' ':
                        break
                    else:
                        a = a + n
                dflog['Unnamed: 7'][0] = a
                dflog['Unnamed: 7'] = dflog['Unnamed: 7'].replace('L', 0)
                dflog['Unnamed: 7'] = dflog['Unnamed: 7'].replace('W', 1)
                dflog['Unnamed: 5'] = dflog['Unnamed: 5'].fillna('away')
                dflog['Unnamed: 5'] = dflog['Unnamed: 5'].replace('@', 'home')

                dflog = dflog.rename(columns={'Unnamed: 5': 'place', 'Unnamed: 7': 'result'})
                dfcl = dfcl.append(dflog, ignore_index=True)
                dfcl.to_excel(cleanpath)
                text1.insert(tk.END, '%s has already update\n' % cleanpath)
            todaylog = todaylog.append(dflog, ignore_index=True)
                
    
            text1.insert(tk.END, 'players have already update\n')
            
    for i in range(len(todaylog)):
        tm = todaylog.iloc[i, :]
        addr = tm[1]
        side1 = os.listdir('C:\\Users\\ecom\\Desktop\\NBAData\\gamelog\\' + seasonyear + '\\East')
        count = 0
        for j in side1:
            if addr == j:
                count = count + 1
        if count > 0:
            side = 'East'
        else:
            side = 'West'
        finpath = 'C:\\Users\\ecom\\Desktop\\NBAData\\gamelog\\'+ seasonyear + '\\' + side + '\\' + addr + '\\home_away.xlsx'
        dffinal = pd.read_excel(finpath)
        dffinaldate = dffinal.Date.max()
        opp = tm[3]
        oppd = todaylog.loc[todaylog.Tm == opp, ]
        opind = oppd.index
        tm[4:24] = tm[4:24] - todaylog.iloc[opind[0],4:24]
        dffinal = dffinal.append(tm)
        dffinal.to_excel(finpath)
        text1.insert(tk.END, '%s home_away has already update\n' % addr)
        
def update_NBAdata():
    today = year + '-' + str(month) + '-' + day
    #check updateable or not
    df = pd.read_excel('C:\\Users\\ecom\\Desktop\\NBAData\\trainingDataSet.xlsx')
    lastday = df.Date.unique().max()
    if lastday >= today:
        text1.delete(1.0, tk.END)
        text1.insert(tk.END, '資料已經最新\n')
    elif lastday < today:
        text1.insert(tk.END, '--------------------------------------------\n')
        text1.insert(tk.END, '開始更新資料\n')
        text1.insert(tk.END, '--------------------------------------------\n')
        #upload data
        updateschedual()
        #取得當天比賽主場隊伍
        colectdata()
        
def colectdata():
    df2 = pd.read_excel('C:\\Users\\ecom\\Desktop\\NBAData\\gamelog\\2007\\East\\ATL\\home_away.xlsx')
    df = pd.DataFrame(columns=df2.columns)
    for i in range(2007,2020):
        mypath = 'C:\\Users\\ecom\\Desktop\\NBAData\\gamelog\\' + str(i)
        side = os.listdir(mypath)
        for j in side:
            mypath = 'C:\\Users\\ecom\\Desktop\\NBAData\\gamelog\\' + str(i) + '\\' + j
            team = os.listdir(mypath)
            for k in team:
                path = 'C:\\Users\\ecom\\Desktop\\NBAData\\gamelog\\' + str(i) + '\\' + j + '\\' + k + '\\home_away.xlsx'
                df1 = pd.read_excel(path)
                df = df.append(df1)
    df.to_excel('C:\\Users\\ecom\\Desktop\\NBAData\\trainingDataSet.xlsx')
    text1.insert(tk.END, 'trainingDataSet has already update\n')
    
    

win = tk.Tk()
win.geometry("900x540")
win.title("運彩分析")
te1 = "歡迎使用運彩分析\n操作方法:\n1.點擊\'分析NBA PK\'按鈕\n2.點擊\'更新NBA資料\'"
text1 = tk.Text(win, font = ("標楷體", 14))
text1.insert(tk.END, te1)
text1.place(x = 40, y = 20, width = 820, height = 390)
# create button
button3 = tk.Button(win, text = "更新model", font = ("標楷體", 13), command = datatraining)
button3.place(x = 484, y = 455, width = 120, height = 40)
button4 = tk.Button(win, text = "分析NBA PK", font = ("標楷體", 13), command = predict_NBA)
button4.place(x = 612, y = 455, width = 120, height = 40)
button5 = tk.Button(win, text = "更新NBA資料", font = ("標楷體", 13), command = update_NBAdata)
button5.place(x = 740, y = 455, width = 120, height = 40)


win.mainloop()

C:\Users\ecom\Anaconda3\lib\site-packages\ipykernel\__main__.py:351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ecom\Anaconda3\lib\site-packages\pandas\core\series.py:749: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._set_with(key, value)


Epoch 1/10
14681/14681 [==============================] - 2s 163us/step - loss: 0.0387 - acc: 0.9891
Epoch 2/10
14681/14681 [==============================] - 2s 131us/step - loss: 0.0345 - acc: 0.9897
Epoch 3/10
14681/14681 [==============================] - 2s 140us/step - loss: 0.0328 - acc: 0.9901
Epoch 4/10
14681/14681 [==============================] - 3s 198us/step - loss: 0.0304 - acc: 0.9917
Epoch 5/10
14681/14681 [==============================] - 3s 171us/step - loss: 0.0294 - acc: 0.9911
Epoch 6/10
14681/14681 [==============================] - 2s 139us/step - loss: 0.0292 - acc: 0.9910
Epoch 7/10
14681/14681 [==============================] - 3s 181us/step - loss: 0.0270 - acc: 0.9916
Epoch 8/10
14681/14681 [==============================] - 2s 161us/step - loss: 0.0271 - acc: 0.9918
Epoch 9/10
14681/14681 [==============================] - 2s 161us/step - loss: 0.0255 - acc: 0.9922
Epoch 10/10
14681/14681 [==============================] - 1s 41us/step
